# serverteeny
(Sabatini's teeny server)

This script runs a server for receiving and executing ssh code. It does the following:\
Receiving:

1. On startup: It establishes an SFTP connection with O2 using input server credentials.
    - For safety, this is best done by setting up an ssh-key. Roughly, in the terminal do:
        - `ssh-keygen -t rsa`
        - `ssh-copy-id username@o2.hms.harvard.edu`
        - `ssh username@o2.hms.harvard.edu`
2. It continuosly searches for a `.json` file with a specific filename (`filename_search`) in a specific directory (`dir_search`). Regular expressions are used to compare the filename. Ex: `run.json`
    - The `.json` file should be a python dictionary with the following fields:
        1. `'name': 'Rich'`     : use the same name every time for logging purposes
        2. `'o2_acct': 'joz608'`     : which user account on o2 to use for sending the ssh commands
        3. `'time_sent': time.ctime()`     : send the current time at sending for logging purposes
        4. `'notes': 'Experiment 3, mouse 5, suite2p'`     : Anything you want to log
        5. `'command': 'python3 /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/dispatcher.py'`     : string to send as a command to o2/slurm
3. If a file is found, it is downloaded to a local directory, and then it loads it in as a python dictionary.
4. It establishes an ssh connection with O2 using the username in the `o2_acct` field.
    - Users/Friends can allow use of the O2 account by following the steps in 1. to store their credentials securely on the computer
5. It sends the command in the `'command'` field
6. It logs the contents of the json file in a `logger.csv` which is stored on O2. This is done by downloading, importing, then uploading again via SFTP.
7. It sends an email to `serverteeny@gmail.com` using SendGrid
8. It cleans up by deleting the `.json` file from remote and closing the SSH object from o2_acct.
9. Go back to step 2

- Note: All temporary files (downloaded `.json` and `.csv` files are kept in `dir_tempFiles_local`)
- In the serverteeny directory there are two subdirectories: `subdirName_run` where `.json` run files are searched for and `subdirName_logger` where the `.csv` file is stored

In [1]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

In [2]:
dir_github = '/media/rich/Home_Linux_partition/github_repos/'
import sys
sys.path.append(dir_github)

%load_ext autoreload
%autoreload 2
from basic_neural_processing_modules import server, email_helpers

In [3]:
from pathlib import Path
import re
import time
from datetime import datetime
import json
import copy

import numpy as np
import pandas as pd

# helpers

# 0. Set settings

In [25]:
fileName_search = 'run.json'
fileName_logger = 'logger.csv'

subdirName_run = 'run'
subdirName_logger = 'logger'

dir_serverteeny_remote = '/n/data1/hms/neurobio/sabatini/serverteeny'
dir_tempFiles_local = '/media/rich/bigSSD/tmp_data/serverteeny_tmpFiles'

pref_sendEmail = True

n_sec_to_wait_between_searches = 10
n_sec_to_wait_after_finding_runFile = 10



dir_run_remote = str(Path(dir_serverteeny_remote) / subdirName_run)

path_loggerCsv_remote = str(Path(dir_serverteeny_remote) / subdirName_logger / 'logger_serverteeny.csv')
path_loggerCsv_local = str(Path(dir_tempFiles_local) / 'logger_serverteeny.csv')

# 1. Establish SFTP+SSH connection with O2

In [5]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')

use_localSshKey = True

pw = server.pw_encode(getpass.getpass(prompt='Password: ')) if use_localSshKey==False else None

path_sshKey = '/home/rich/.ssh/id_rsa' if use_localSshKey else None

Username: rh183


In [6]:
## initialize ssh_compute
ssh_scraper = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_scraper.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)
sftp = server.sftp_interface(ssh_client=ssh_scraper.client)

Find or make the remote serverteeny directory

In [7]:
def mkdir_verbose_remote(directory, dir_name=''):
    found_directory = sftp.isdir_remote(directory)
    if found_directory:
        print(f'found remote {dir_name} directory:  {directory}')
    else:
        print(f'making remote {dir_name} directory:  {directory}')
        sftp.mkdir_safe(directory)

def mkdir_verbose_local(directory, dir_name=''):
    found_directory = Path(directory).is_dir()
    if found_directory:
        print(f'found local {dir_name} directory:  {directory}')
    else:
        print(f'making local {dir_name} directory:  {directory}')
        Path(directory).mkdir(parents=True, exist_ok=True)

In [8]:
mkdir_verbose_remote(dir_serverteeny_remote, 'serverteeny')
mkdir_verbose_remote(dir_run_remote, 'serverteeny run')
mkdir_verbose_remote(str(Path(path_loggerCsv_remote).parent), 'serverteeny logger')

mkdir_verbose_local(dir_tempFiles_local, 'serverteeny temp')

found remote serverteeny directory:  /n/data1/hms/neurobio/sabatini/serverteeny
found remote serverteeny run directory:  /n/data1/hms/neurobio/sabatini/serverteeny/run
found remote serverteeny logger directory:  /n/data1/hms/neurobio/sabatini/serverteeny/logger
found local serverteeny temp directory:  /media/rich/bigSSD/tmp_data/serverteeny_tmpFiles


# 2. Search for `.json` run file

In [26]:
def search_for_first_match_remote(
    dir_remote,
    fileName,
    verbose=True,
    n_sec_to_wait_between_searches=10,
    n_sec_to_wait_after_finding_runFile=10,
):
    """
    Searches for fileName in dir_remote.
    RH 2022
    """

    found=False

    while found==False:
        contents = np.array(sftp.sftp.listdir(dir_remote))
        comparisons = np.array([re.search(fileName, c) is not None for c in contents])
        if sum(comparisons) > 0:
            path_match = str(Path(dir_remote) / contents[comparisons][0])  ## just take the first index in case there are multiple matches
            print(f'found match:  {path_match}')
            
            print(f'pausing for {n_sec_to_wait_after_finding_runFile} seconds')
            time.sleep(n_sec_to_wait_after_finding_runFile)
            
            return path_match
        
        time.sleep(n_sec_to_wait_between_searches)

In [ ]:
path_runFile_remote = search_for_first_match_remote(
    dir_run_remote,
    fileName_search,
    verbose=True,
    n_sec_to_wait_between_searches=n_sec_to_wait_between_searches,
    n_sec_to_wait_after_finding_runFile=n_sec_to_wait_after_finding_runFile,
)

# 3. Download `.json` run file

In [11]:
path_runFile_local = str(Path(dir_tempFiles_local) / Path(path_runFile_remote).name)

print(f'downloading .json run file from  {path_runFile_remote}  to  {path_runFile_local}')
sftp.sftp.get(
    remotepath=path_runFile_remote,
    localpath=path_runFile_local,
)

downloading .json run file from  /n/data1/hms/neurobio/sabatini/serverteeny/run/run.json  to  /media/rich/bigSSD/tmp_data/serverteeny_tmpFiles/run.json


In [12]:
with open(path_runFile_local, mode='r') as f:
    runFile = json.load(f)

In [15]:
# runFile = {
#     'name': 'Rich',
#     'o2_acct': 'rh183',
#     'command': 'python3 /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/dispatcher.py',
#     'notes': 'Experiment 3, mouse 5, suite2p',
#     'time_sent': time.ctime(),
# }

# 4. Log into `o2_acct` using saved O2 account credentials

In [16]:
## initialize ssh_compute
ssh_c = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=runFile['o2_acct'],
    password=None,
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

Exception ignored in: <function ssh_interface.__del__ at 0x7f45f95631f0>
Traceback (most recent call last):
  File "/media/rich/Home_Linux_partition/github_repos/basic_neural_processing_modules/server.py", line 395, in __del__
    self.ssh.close()
AttributeError: 'NoneType' object has no attribute 'close'


# 5. Send command

In [17]:
ssh_c.send_receive('', timeout=1, verbose=True)
ssh_c.send(runFile['command'])
recv_command = ssh_c.expect(str_success=f"[{runFile['o2_acct']}", total_timeout=1)
ssh_c.send_receive('', timeout=1, verbose=True);


(base) [rh183@login02 ~]$ 
aceRhythm/dispatcher.pyurobio/sabatini/rich/analysis/f 

python3: can't open file '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/dispatcher.py': [Errno 2] No such file or directory
(base) [rh183@login02 ~]$ 

(base) [rh183@login02 ~]$ 


# 6. Log `.json` run file contents to `.csv` logger file

download master copy of `logger.csv` file from remote

In [18]:
def load_logger(path_logger):
    return pd.read_csv(
        filepath_or_buffer=path_logger,
#         sep=NoDefault.no_default,
        delimiter=None, 
        header='infer', 
#         names=NoDefault.no_default,
        index_col=None,
        usecols=None,
        squeeze=None,
#         prefix=NoDefault.no_default,
        mangle_dupe_cols=True,
        dtype=None, 
    )

Add a couple things to make an augmented version of the runFile

In [19]:
runFile_aug = copy.deepcopy(runFile)
runFile_aug['time_run'] = time.ctime()
runFile_aug['datetime_Ymd_HMS_f'] = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
runFile_aug['cmd_receipt'] = recv_command[0]

try to find the logger on remote and download it. if it's there, append the augmented runFile to it. if its not there, make a new local one.

In [20]:
try:
    sftp.sftp.get(
        remotepath=path_loggerCsv_remote,
        localpath=path_loggerCsv_local,
    )  ## note that when this fails, it still makes an empty local file with the correct path    
    print(f'found and downloaded remote copy of logger. from:  {path_loggerCsv_remote}  to:  {path_loggerCsv_local}')
    
    logger = pd.read_csv(filepath_or_buffer=path_loggerCsv_local)
    pd.DataFrame(runFile_aug, index=[logger.shape[0]]).to_csv(path_loggerCsv_local, mode='a', header=False) ## append the new augmented runFile to the end of it. make the index the existing number of entries in the logger
    print(f'appended run file dictionary to local copy of logger')
    
except:
    print(f'Failed to retrieve logger from remote. Making new one here:  {path_loggerCsv_local}')

    pd.DataFrame(runFile_aug, index=[0]).to_csv(path_loggerCsv_local, mode='w')  ## write a new logger

## upload the updated or new logger file to remote
sftp.sftp.put(
    localpath=path_loggerCsv_local,
    remotepath=path_loggerCsv_remote,
);
print(f'uploaded updated or new logger file to remote')

found and downloaded remote copy of logger. from:  /n/data1/hms/neurobio/sabatini/serverteeny/logger/logger_serverteeny.csv  to:  /media/rich/bigSSD/tmp_data/serverteeny_tmpFiles/logger_serverteeny.csv
appended run file dictionary to local copy of logger
uploaded updated or new logger file to remote


# 7. Send email with run file information

In [21]:
def dict_to_str(dictionary):
    return '\n'.join([f'{key}:  {val}' for key,val in dictionary.items()])

In [22]:
if pref_sendEmail:
#     email_body = 
    sender = email_helpers.Sender(api_key='')
    sender.send(
            from_email='serverteeny@gmail.com', 
            to_emails ='serverteeny@gmail.com', 
            subject='Run log', 
            content=dict_to_str(runFile_aug),
            verbose=True,
    )

202
b''
Server: nginx
Date: Sat, 04 Jun 2022 09:08:28 GMT
Content-Length: 0
Connection: close
X-Message-Id: 7PA3yMXyRd-Sl8YPVOx2dg
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html
Strict-Transport-Security: max-age=600; includeSubDomains




# 8. Delete the remote `.json` run file

In [23]:
try:
    sftp.sftp.remove(path_runFile_remote)
    print('deleted remote .json run file')
except:
    print("couldn't delete remote .json file")

deleted remote .json run file


In [24]:
ssh_c.close()
del ssh_c